In [8]:
from glob import glob
import os
from collections import defaultdict
from pathlib import Path
from enum import Enum
import json
import pandas as pd
import seaborn as sns

In [9]:
UNATES_LOOKUP_PATH = '../experiments/find-unates/'

# Load Experiment Results

In [10]:
class Status(Enum):
    ERROR = "Error"
    SUCCESS = "Success"
    NOT_FOUND = "Not Found"
    TIMEOUT = "Timeout"
    
    def __repr__(self):
        return str(self.value)
    
    def __str__(self):
        return str(self.value)

In [11]:
def get_all_benchmarks(path):
    hoa_files = glob(os.path.join(path, "*.json"))
    names = [
        Path(f).stem
        for f in hoa_files
    ]
    return names

print("Total benchmark of Unates" , len(get_all_benchmarks(UNATES_LOOKUP_PATH)))

Total benchmark of Unates 207


In [12]:
def extract_vars_from_str(x):
    if len(x) == 0:
        return []
    return x.split(",")

In [13]:
def benchmark_unate_loader(path: str):
    def load_benchmark(name: str):
        base = {
            'Name': name,
        }
        
        json_path = os.path.join(path, name + ".json")
        
        if not os.path.exists(json_path):
            return {
                **base,
                'Status': Status.NOT_FOUND
            }
        
        with open(json_path, 'r') as f:
            content = f.readlines()
            try:
                benchmark_json = json.loads(content[0])
            except Exception as e:
                return {
                    **base,
                    'Status': Status.ERROR,
                    'Error': content
                }
        
        output_vars = benchmark_json['output_vars']
        is_completed = benchmark_json['is_completed']
        total_duration = benchmark_json['total_time']
        is_automaton_build = benchmark_json['automaton']['is_built']
        
        if not is_automaton_build:
            return {
                **base,
                'Status': Status.TIMEOUT,
                'Is Completed': is_completed,
            }
        
        automaton_build_duration = benchmark_json['automaton']['build_duration']
        total_states = benchmark_json['automaton']['total_states']
        total_output_vars = len(benchmark_json['output_vars'])
        
        vars_positive_unates = { var: 0 for var in output_vars }
        vars_negative_unates = { var: 0 for var in output_vars }
        state_positive_unates = { state: 0 for state in range(total_states) }
        state_negative_unates = { state: 0 for state in range(total_states) }
        states_complement_duration = { state: 0 for state in range(total_states) }
        states_removed_edges = { state: 0 for state in range(total_states) }
        states_impacted_edges = { state: 0 for state in range(total_states) }
        unate_variables_after_retest = set()
        
        for state_unateness in benchmark_json['unate_states']:
            state = state_unateness['state']
            state_positive_vars = extract_vars_from_str(state_unateness['positive_unate_variables'])
            state_negative_vars = extract_vars_from_str(state_unateness['negative_unate_variables'])
            not_unate_variables = extract_vars_from_str(state_unateness['not_unate_variables'])
            assert set(state_positive_vars).isdisjoint(set(state_negative_vars))
            
            variables_after_retest = set(state_positive_vars).union(set(state_negative_vars)).intersection(set(not_unate_variables))
            unate_variables_after_retest.update(variables_after_retest)
            
            states_removed_edges[state] = state_unateness['removed_edges']
            states_impacted_edges[state] = state_unateness['impacted_edges']
            
            states_complement_duration[state] = state_unateness['complement_duration']
            
            for var in state_positive_vars:
                vars_positive_unates[var] += 1
                state_positive_unates[state] += 1
            
            for var in state_negative_vars:
                vars_negative_unates[var] += 1
                state_negative_unates[state] += 1
        
        state_total_unates = { state: state_positive_unates[state] + state_negative_unates[state] for state in range(total_states) }
        vars_positive_unates_percentage = { var: (count / total_states) * 100 for var, count in vars_positive_unates.items() }
        vars_negative_unates_percentage = { var: (count / total_states) * 100 for var, count in vars_negative_unates.items() }
        state_positive_unates_percentage = { state: (count / total_output_vars) * 100 for state, count in state_positive_unates.items() }
        state_negative_unates_percentage = { state: (count / total_output_vars) * 100 for state, count in state_negative_unates.items() }
        
        """
        * Convert MS to Seconds / Minutes / Etc

        Expecting to include:
        1. For each variable:
            1.1 how many % of the states it is Unate on (Positive, Negative)
        2. For each state:
            2.1 how long it took to check Unateness
            2.2 What's the duration of complementing the state
            2.3 How many edges are impacted?
            2.4 How many edges are removed?
            2.5 How many positive and negative unates it has
            2.6 How many Unate variable was found after we retested the variable
        
        3. Automaton
            3.1 How long it took to build the automaton
            3.2 How many states in the automaton
        
        4. Total duration of the process
        5. How long it took to search for unates
        6. How many output variables?
        7. How many totally Unate variables there are (Positive and Negative)?
        """
        
        return {
            # Group 1: General
            **base,
            'Status': Status.SUCCESS,
            'Is Completed': is_completed,
            'Total Duration': total_duration,
            
            # Group 2: Automaton
            'Is Automaton Built': is_automaton_build,
            'Automaton Build Duration': automaton_build_duration,
            'Total States': total_states,
            'Total Output Vars': total_output_vars,
            'Total Impacted Edges': sum(states_impacted_edges.values()),
            'Total Removed Edges': sum(states_removed_edges.values()),
            'Impacted Edges': states_impacted_edges,
            'Removed Edges': states_removed_edges,

            # Group 3: Unateness by Vars
            'Vars Positive Unates Percentage': vars_positive_unates_percentage,
            'Vars Negative Unates Percentage': vars_negative_unates_percentage,
            'Total Vars Positive Unates': len(vars_positive_unates),
            'Total Vars Negative Unates': len(vars_negative_unates),
            'Vars Positive Unates (At Least One State)': [ var for var, count in vars_positive_unates.items() if count > 0 ],
            'Vars Negative Unates (At Least One State)': [ var for var, count in vars_negative_unates.items() if count > 0 ],
            'Vars Positive Unates (All States)': [ var for var, count in vars_positive_unates.items() if count == total_states ],
            'Vars Negative Unates (All States)': [ var for var, count in vars_negative_unates.items() if count == total_states ],
            'Vars Positive Unates': vars_positive_unates,
            'Vars Negative Unates': vars_negative_unates,
            'Found Unate when Retested': list(unate_variables_after_retest),

            # Group 4: Unateness by State
            'States Positive Unates Percentage': state_positive_unates_percentage,
            'States Negative Unates Percentage': state_negative_unates_percentage,
            'States with 100% Unates': [ state for state, count in state_total_unates.items() if count == total_output_vars ],
            'Total States Positive Unates': len(state_positive_unates),
            'Total States Negative Unates': len(state_negative_unates),
            'States Positive Unates': state_positive_unates,
            'States Negative Unates': state_negative_unates,
            'States Total Unates': state_total_unates,
            'States Complement Total Duration': len(states_complement_duration.values()),
            'States Complement Duration': states_complement_duration,
        }

    return load_benchmark

In [14]:
unates_loader = benchmark_unate_loader(UNATES_LOOKUP_PATH)
unates_df = pd.DataFrame([
    unates_loader(name)
    for name in get_all_benchmarks(UNATES_LOOKUP_PATH)
])
unates_df.to_csv('unates.csv', index=False)
unates_df.head(5)

,Name,Status,Is Completed,Total Duration,Is Automaton Built,Automaton Build Duration,Total States,Total Output Vars,Total Impacted Edges,Total Removed Edges,...,States Negative Unates Percentage,States with 100% Unates,Total States Positive Unates,Total States Negative Unates,States Positive Unates,States Negative Unates,States Total Unates,States Complement Total Duration,States Complement Duration,Error
0,ltl2dpa20,Success,True,21.0,True,12.0,10.0,2.0,0.0,0.0,...,"{0: 0.0, 1: 0.0, 2: 50.0, 3: 50.0, 4: 0.0, 5: ...","[2, 3]",10.0,10.0,"{0: 0, 1: 0, 2: 1, 3: 1, 4: 0, 5: 0, 6: 0, 7: ...","{0: 0, 1: 0, 2: 1, 3: 1, 4: 0, 5: 0, 6: 0, 7: ...","{0: 0, 1: 0, 2: 2, 3: 2, 4: 0, 5: 0, 6: 0, 7: ...",10.0,"{0: 7, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ...",NaN
1,TwoCountersInRangeA2,Success,True,36420.0,True,4529.0,6.0,8.0,0.0,0.0,...,"{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0}",[],6.0,6.0,"{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}","{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}","{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}",6.0,"{0: 15375, 1: 3226, 2: 3168, 3: 3385, 4: 3224,...",NaN
2,lilydemo24,Success,True,31.0,True,12.0,14.0,4.0,14.0,2.0,...,"{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 25...","[1, 3, 5]",14.0,14.0,"{0: 3, 1: 4, 2: 0, 3: 4, 4: 0, 5: 3, 6: 0, 7: ...","{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: ...","{0: 3, 1: 4, 2: 0, 3: 4, 4: 0, 5: 4, 6: 0, 7: ...",14.0,"{0: 6, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ...",NaN
3,OneCounterGuiA9,Success,True,124683.0,True,11030.0,9.0,9.0,0.0,0.0,...,"{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0....",[],9.0,9.0,"{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ...","{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ...","{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ...",9.0,"{0: 26335, 1: 2328, 2: 2424, 3: 2417, 4: 70763...",NaN
4,ltl2dba24,Success,True,7.0,True,5.0,4.0,1.0,4.0,2.0,...,"{0: 0.0, 1: 0.0, 2: 100.0, 3: 0.0}","[0, 1, 2]",4.0,4.0,"{0: 1, 1: 1, 2: 0, 3: 0}","{0: 0, 1: 0, 2: 1, 3: 0}","{0: 1, 1: 1, 2: 1, 3: 0}",4.0,"{0: 1, 1: 0, 2: 0, 3: 0}",NaN


# Header Groups
- General: Name | Status | Is Completed | Total Duration | Error
- Automaton: Is Automaton Built | Automaton Build Duration | Total States | Total Output Vars | Total Impacted Edges | Total Removed Edges | Impacted Edges | Removed Edges
- Unates By Vars: Vars Positive Unates Percentage | Vars Negative Unates Percentage | Total Vars Positive Unates | Total Vars Negative Unates | Vars Positive Unates (At Least One State) | Vars Negative Unates (At Least One State) | Vars Positive Unates (All States) | Vars Negative Unates (All States) | Vars Positive Unates | Vars Negative Unates
- Unates by States: States Positive Unates Percentage | States Negative Unates Percentage | States with 100% Unates | Total States Positive Unates | Total States Negative Unates | States Positive Unates | States Negative Unates | States Total Unates | States Complement Duration